##### Build Custom Dataset

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from datetime import timedelta, date, datetime

In [ ]:
class PriceDataset(Dataset):
    def __init__(self, item, timespan, start_date_str, end_date_str):
        self.directory = 'csvfiles'
        self.item = item
        self.timespan = timespan
        start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = datetime.strptime(end_date_str, '%Y-%m-%d').date()
        self.dates = [single_date.strftime("%Y-%m-%d") for single_date in self.daterange(start_date, end_date)]
        self.columns = [0, 1, 2, 3, 4, 7]

    def daterange(self, start_date, end_date):
        for n in range(int((end_date - start_date).days) + 1):
            yield start_date + timedelta(n)

    def __len__(self):
        return len(self.dates)

    def __getitem__(self, idx):
        date = self.dates[idx]
        filename = f"{self.directory}/{self.item}-{self.timespan}-{date}.csv"
        df = pd.read_csv(filename, usecols=self.columns, header=None)
        return torch.tensor(df.values, dtype=torch.float32)

##### Set Dataset and DataLoader
* Send Dataset and DataLoader to GPU for faster Calculation

In [ ]:
dataset = PriceDataset('BTCUSDT', '1m', '2021-03-01', '2023-04-30')
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

##### Build LSTM Model

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from pandas import DataFrame as df

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=4, hidden_layer_size=100, output_size=2):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]